# Testing ChatESG

In [20]:
import requests
import json

In [21]:
TEST_UID = "#test-uid"
MODE = "local"

if MODE=="local":
    base_url = 'http://localhost:8000'

# Health check
requests.get(f"{base_url}/health").content, base_url

(b'{"status":"OK"}', 'http://localhost:8000')

### Chat

In [63]:
headers_chat = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

chat_payload = {
    "src": "skripsi",
    "model": "gpt-4o-mini",
    # "model": 'climategpt-13b',
    # "bsid": "7f79f395c3564549be52",
    "retriever": 'similarity',
    "top_k": 1,
    "inputs": [
        # {"type": "text", "role": "user", "content": "sebutkan nama presiden komisaris dari perusahaan"}
        {"type": "text", "role": "user", "content": "berapa jumlah karyawan berdasarkan gender?"}
    ]
}

response = requests.post(f"{base_url}/api/v1/chat", headers=headers_chat, data=json.dumps(chat_payload), stream=True)
for line in response.iter_lines():
    if line:
        chunk = json.loads(line.decode('utf-8'))
        print("Chunk received:", chunk)
chunk

Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': [], 'sources': None, 'bsid': 'c3a7858b520045dbb3f3', 'bcid': 'f4aed020-57fa-4d9d-8de3-85e1bc0aec04', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': [''], 'sources': None, 'bsid': 'c3a7858b520045dbb3f3', 'bcid': 'f4aed020-57fa-4d9d-8de3-85e1bc0aec04', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': ['Jumlah'], 'sources': None, 'bsid': 'c3a7858b520045dbb3f3', 'bcid': 'f4aed020-57fa-4d9d-8de3-85e1bc0aec04', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': ['Jumlah k'], 'sources': None, 'bsid': 'c3a7858b520045dbb3f3', 'bcid': 'f4aed020-57fa-4d9d-8de3-85e1bc0aec04', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Menga

{'done': True,
 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'],
 'responses': ['Jumlah karyawan berdasarkan gender di PT Astra International Tbk pada tahun 2023 adalah sebagai berikut:\n\n- Pria (Male): 181,795\n- Wanita (Female): 19,758\n\nTotal karyawan: 201,553.'],
 'sources': [{'url': 'https://www.astra.co.id/',
   'pages': [115],
   'snippet': None,
   'related_indicators': None}],
 'bsid': 'c3a7858b520045dbb3f3',
 'bcid': 'f4aed020-57fa-4d9d-8de3-85e1bc0aec04',
 'metadata': None}

### Retrieval

In [70]:
headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

payload = {
    "query": "Berapa jumlah karyawan perusahaan berdasarkan gender?",
    "filter": {"company": "PT Astra International Tbk", "year": 2022},
    "mode": "indicator-cls",
    "top_k": 4,
    "model": "gpt-4o-mini"
}

response = requests.post(f"{base_url}/api/v1/retrieval", headers=headers, data=json.dumps(payload))
json.loads(response.content.decode('utf-8'))

{'contents': [{'id': None,
   'metadata': {'company': 'PT Astra International Tbk',
    'disclosed_gri': ['405-1'],
    'page': 114.0,
    'ticker': 'ASII',
    'year': 2022.0},
   'page_content': "## Woman Leaders Program\n\nSepanjang tahun 2023, Astra melaksanakan sejumlah aktivitas untuk memperkuat keberagaman dan inklusi yang sejalan dengan Astra 2030 Sustainability Aspirations, seperti:\n\nThroughout 2023, Astra carried out numerous acitivities to strengthen diversity and inclusion in line with the Astra 2030 Sustainability Aspirations, namely:\n\n## DIVERSITY & INCLUSION FOR LEADERS\n\n## IGNITE THE IMPACT OF WOMEN LEADERS\n\n23 batch dengan total 605 peserta manajemen eksekutif dari 94 perusahaan Grup Astra\n\n23 batch with 605 participants from 94 companies across Astra Group\n\n## ASTRA WOMEN SUPPORT SYSTEM\n\n22 sesi mentoring dengan 11 eksekutif wanita dari 11 perusahaan\n\n22 mentoring sessions covering 11 women executives from 11 companies\n\nBRANDING & EDUCATIONAL CONTENT

### Rouge

In [83]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import RougeScore

sample = SingleTurnSample(
    response="Berdasarkan laporan keberlanjutan PT Astra International Tbk tahun 2022, total limbah yang diarahkan untuk pembuangan pada tahun 2022 adalah 24.55 ton untuk limbah padat non-B3 dan 3.38 ton untuk limbah padat B3. Total berat limbah yang diarahkan untuk pembuangan pada tahun 2022 adalah 24.55 + 3.38 = **27.93 ton**.",
    reference="Total berat limbah yang diarahkan untuk pembuangan oleh organisasi dalam tahun 2022 adalah 51,267.71 ribu ton, terdiri dari 51,239.78 ribu ton limbah cair dan 27.93 ribu ton limbah padat."
)

scorer = RougeScore(rouge_type='rouge1', measure_type='recall')
await scorer.single_turn_ascore(sample)

0.5588235294117647

### Factual Correctness

In [40]:
response_text="""
Berdasarkan laporan keberlanjutan PT Astra International Tbk tahun 2022, berikut adalah asosiasi industri, asosiasi keanggotaan lain, serta organisasi advokasi nasional atau internasional yang diikuti oleh Astra, di mana organisasi tersebut memiliki peran penting:

### Asosiasi Industri:
1. **Asosiasi Industri Sepeda Motor Indonesia (AISI)**
2. **Gabungan Industri Alat Mobil dan Motor (GIAMM)**
3. **Gabungan Industri Kendaraan Bermotor Indonesia (GAIKINDO)**
4. **Asosiasi Asuransi Jiwa Indonesia (AAJI)**
5. **Asosiasi Asuransi Syariah Indonesia (AASI)**
6. **Asosiasi Asuransi Umum Indonesia (AAUI)**
7. **Asosiasi Jasa Konstruksi Nasional (GAPENSI)**
8. **Asosiasi Kontraktor Indonesia (AKI)**
9. **Perhimpunan Agen Alat Berat Indonesia (PAABI)**
10. **Gabungan Perusahaan Kelapa Sawit Indonesia (GAPKI)**
11. **Himpunan Masyarakat Gambut Indonesia (HGI)**
12. **Asosiasi Jalan Tol Indonesia (ATI)**
13. **Asosiasi Logistik & Forwarder Indonesia (ALFI)**
14. **Asosiasi Perusahaan Rental Kendaraan Indonesia (ASPERKINDO)**
15. **Indonesian National Shipowners' Association (INSA)**
16. **Persatuan Balai Lelang Indonesia (PERBALI)**
17. **Asosiasi Industri Teknologi Informasi (AITI)**
18. **Asosiasi Pengusaha Komputer Indonesia (APKOMINDO)**
19. **Asosiasi Emiten Indonesia (AEI)**
20. **Asosiasi Pengusaha Indonesia (APINDO)**
21. **Kamar Dagang dan Industri Indonesia (KADIN)**

### Organisasi Advokasi dan Konsorsium:
1. **Forum Kemitraan Pembangunan Sosial Suku Anak Dalam (FKPSAD)**
2. **Konsorsium Genom Seluruh Indonesia**
3. **Palm Oil Genome Oil Project**

### Keanggotaan Lain:
1. **Asosiasi Perusahaan Perdagangan Barang Distributor, Keagenan dan Industri Indonesia (ARDIN INDONESIA)**
2. **Asosiasi Perusahaan Pengadaan Komputer dan Telematika Indonesia (ASPEKMI)**
3. **Ikatan Akuntan Indonesia (IAI)**
4. **Indonesia Corporate Secretary Association (ICSA)**

Keterlibatan Astra dalam berbagai asosiasi ini menunjukkan komitmennya untuk berkolaborasi dan berkontribusi dalam pengembangan industri serta advokasi isu-isu yang relevan di tingkat nasional dan internasional.
"""
reference_text="""
PT Astra International aktif terlibat dalam berbagai asosiasi industri dan organisasi, antara lain:

1. **Asosiasi Industri:**
   - Asosiasi Industri Sepeda Motor Indonesia (AISI)
   - Gabungan Industri Alat Mobil dan Motor (GIAMM)
   - Gabungan Industri Kendaraan Bermotor Indonesia (GAIKINDO)
   - Gabungan Perusahaan Kelapa Sawit Indonesia (GAPKI)
   - Asosiasi Jasa Konstruksi Nasional (GAPENSI)

2. **Asosiasi Keanggotaan Lain:**
   - Asosiasi Asuransi Jiwa Indonesia (AAJI)
   - Asosiasi Asuransi Syariah Indonesia (AASI)
   - Asosiasi Emiten Indonesia (AEI)
   - Kamar Dagang dan Industri Indonesia (KADIN)

3. **Organisasi Advokasi:**
   - Forum Kemitraan Pembangunan Sosial Suku Anak Dalam (FKPSAD) / Orang Rimba Social Development Partnership Forum
   - Konsorsium Genom Seluruh Indonesia
   - Palm Oil Genome Oil Project

Keterlibatan dalam organisasi-organisasi ini menunjukkan peran aktif Astra dalam membangun jaringan dan berkontribusi pada perkembangan industri serta advokasi sosial.
"""

In [41]:
headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

payload = {
    "response": response_text,
    "reference": reference_text,
    "model_name": "gpt-4o-mini",
}

response = requests.post(f"{base_url}/api/v1/eval/fc_metric", headers=headers, data=json.dumps(payload))
json.loads(response.content.decode('utf-8'))

{'score': 0.9999999991666666,
 'response_claims': ['PT Astra International Tbk published a sustainability report in 2022.',
  'PT Astra International Tbk is a member of the Indonesian Motorcycle Industry Association (AISI).',
  'PT Astra International Tbk is a member of the Indonesian Automotive Equipment and Motor Industry Association (GIAMM).',
  'PT Astra International Tbk is a member of the Indonesian Automotive Industry Association (GAIKINDO).',
  'PT Astra International Tbk is a member of the Indonesian Life Insurance Association (AAJI).',
  'PT Astra International Tbk is a member of the Indonesian Sharia Insurance Association (AASI).',
  'PT Astra International Tbk is a member of the Indonesian General Insurance Association (AAUI).',
  'PT Astra International Tbk is a member of the National Construction Services Association (GAPENSI).',
  'PT Astra International Tbk is a member of the Indonesian Contractors Association (AKI).',
  'PT Astra International Tbk is a member of the In